In [23]:
import numpy as np
import random
from collections import namedtuple
from itertools import combinations
import matplotlib.pyplot as plt

## Common tests

In [24]:
problem = np.load('lab2/problem_r1_500.npy')
num_cities = problem.shape[0]

In [25]:
PROBLEM_SIZE = num_cities
Individual = namedtuple('Individual', ['genotype', 'fitness'])

In [26]:
# return the list of distances according to the current solution
def get_distances(current_sol):
    distances = []
    for idx, city_a in enumerate(current_sol):
        city_b_idx = idx + 1
        if city_b_idx == len(current_sol):  # Also fixed this
            city_b_idx = 0

        city_b = current_sol[city_b_idx]
        
        d = problem[city_a, city_b]
        distances.append(d)
    
    return distances

# return the total distance for the current solution
def tot_distance(distances):
    return sum(distances)

# Evaluate a tour and return fitness
def evaluate_tour(tour):
    return tot_distance(get_distances(tour))


In [27]:
def tournament_selection(population, tau=5):
    """Tournament selection - good for minimization"""
    pool = random.sample(population, min(tau, len(population)))
    return min(pool, key=lambda i: i.fitness)

def rank_selection(population):
    """Rank-based selection - works well for minimization"""
    sorted_pop = sorted(population, key=lambda i: i.fitness)
    ranks = list(range(len(sorted_pop), 0, -1))  # Best gets highest rank
    total_rank = sum(ranks)
    weights = [r / total_rank for r in ranks]
    return random.choices(sorted_pop, weights=weights)[0]

def uniform_selection(population):
    """Uniform (random) selection - all individuals have equal probability"""
    return random.choice(population)

In [28]:
# MUTATIONS
def inversion_mutation(genotype):
    """Reverse a random segment of the tour"""
    new_genotype = genotype[:]
    idx1, idx2 = sorted(random.sample(range(len(genotype)), 2))
    new_genotype[idx1:idx2+1] = reversed(new_genotype[idx1:idx2+1])
    return new_genotype


def scramble_mutation(genotype):
    """Scramble a random segment of the tour"""
    new_genotype = genotype[:]
    idx1, idx2 = sorted(random.sample(range(len(genotype)), 2))
    segment = new_genotype[idx1:idx2+1]
    random.shuffle(segment)
    new_genotype[idx1:idx2+1] = segment
    return new_genotype

def insert_mutation(genotype):
    """Remove a city and insert it at a random position"""
    new_genotype = genotype[:]
    idx1, idx2 = random.sample(range(len(genotype)), 2)
    city = new_genotype.pop(idx1)
    new_genotype.insert(idx2, city)
    return new_genotype


In [29]:
# CROSSOVERS

def one_cut_xover(p1: Individual, p2: Individual) -> Individual:
    idx = random.randint(0, PROBLEM_SIZE - 1)
    new_genotype = p1.genotype[:idx] + p2.genotype[idx:]
    return Individual(new_genotype, None)

def order_crossover(p1, p2):
    """Order Crossover (OX1) - preserves relative order"""
    size = len(p1)
    start, end = sorted(random.sample(range(size), 2))
    
    child = [None] * size
    child[start:end] = p1[start:end]
    
    p2_cities = [city for city in p2 if city not in child]
    pos = 0
    for i in range(size):
        if child[i] is None:
            child[i] = p2_cities[pos]
            pos += 1
    
    return child

def partially_mapped_crossover(p1, p2):
    """Partially Mapped Crossover (PMX)"""
    size = len(p1)
    start, end = sorted(random.sample(range(size), 2))
    
    child = [None] * size
    child[start:end] = p1[start:end]
    
    # Create mapping
    for i in range(start, end):
        if p2[i] not in child:
            pos = i
            while start <= pos < end:
                pos = p2.index(p1[pos])
            child[pos] = p2[i]
    
    # Fill remaining
    for i in range(size):
        if child[i] is None:
            child[i] = p2[i]
    
    return child

def edge_recombination_crossover(p1, p2):
    """Edge Recombination Crossover - preserves edges"""
    size = len(p1)
    
    # Build adjacency list
    edges = {i: set() for i in range(num_cities)}
    for tour in [p1, p2]:
        for i in range(size):
            curr = tour[i]
            next_city = tour[(i + 1) % size]
            prev_city = tour[(i - 1) % size]
            edges[curr].add(next_city)
            edges[curr].add(prev_city)
    
    child = []
    current = random.choice([p1[0], p2[0]])
    child.append(current)
    
    while len(child) < size:
        # Remove current from all edge lists
        for city in edges:
            edges[city].discard(current)
        
        # Find next city
        neighbors = edges[current]
        if neighbors:
            # Choose neighbor with fewest edges
            next_city = min(neighbors, key=lambda x: len(edges[x]))
        else:
            # Choose random unvisited city
            unvisited = [c for c in range(num_cities) if c not in child]
            if not unvisited:
                break
            next_city = random.choice(unvisited)
        
        child.append(next_city)
        current = next_city
    
    return child

def cycle_crossover(p1, p2):
    """Cycle Crossover (CX)"""
    size = len(p1)
    child = [None] * size
    
    while None in child:
        idx = child.index(None)
        start_idx = idx
        
        while True:
            child[idx] = p1[idx]
            idx = p2.index(p1[idx])
            if idx == start_idx:
                break
        
        p1, p2 = p2, p1
    
    return child

In [30]:
def random_initialization(size, n):
    """Random permutations"""
    population = []
    for _ in range(n):
        genotype = list(range(size))
        random.shuffle(genotype)
        fitness = evaluate_tour(genotype)
        population.append(Individual(genotype, fitness))
    return population

def nearest_neighbor_initialization(size, n):
    """Greedy nearest neighbor heuristic"""
    population = []
    
    for _ in range(n):
        start = random.randint(0, size - 1)
        tour = [start]
        unvisited = set(range(size)) - {start}
        
        while unvisited:
            current = tour[-1]
            # Find nearest unvisited city
            nearest = min(unvisited, key=lambda c: problem[current, c])
            tour.append(nearest)
            unvisited.remove(nearest)
        
        fitness = evaluate_tour(tour)
        population.append(Individual(tour, fitness))
    
    return population

def mixed_initialization(size, n):
    """Mix of random and greedy initialization"""
    population = []
    # 30% greedy, 70% random
    greedy_count = int(n * 0.3)
    population.extend(nearest_neighbor_initialization(size, greedy_count))
    population.extend(random_initialization(size, n - greedy_count))
    return population

In [31]:
def evolution_strategy_comma(config):
    """(μ, λ) Evolution Strategy - offspring replace parents"""
    print(f"\n{'='*60}")
    print(f"EVOLUTION STRATEGY (μ, λ)")
    print(f"{'='*60}")
    
    mu = config['pop_size']
    lambda_ = config['offspring_size']
    
    # Initialize
    population = config['init_func'](num_cities, mu)
    best_ever = min(population, key=lambda i: i.fitness)
    history = [best_ever.fitness]
    
    for gen in range(config['max_gen']):
        offspring = []
        
        # Generate λ offspring from μ parents
        for _ in range(lambda_):
            # Recombination
            if random.random() < 0.7:
                p1, p2 = random.sample(population, 2)
                child_genotype = config['crossover'](p1.genotype, p2.genotype)
            else:
                parent = random.choice(population)
                child_genotype = parent.genotype[:]
            
            # Mutation (always applied in ES)
            if random.random() < 0.5:
                child_genotype = config['mutation'](child_genotype)
            else:
                child_genotype = scramble_mutation(child_genotype)
            
            fitness = evaluate_tour(child_genotype)
            offspring.append(Individual(child_genotype, fitness))
        
        # Select best μ from λ offspring (comma selection)
        population = sorted(offspring, key=lambda i: i.fitness)[:mu]
        
        current_best = population[0]
        if current_best.fitness < best_ever.fitness:
            best_ever = current_best
        
        history.append(best_ever.fitness)
        
        if (gen + 1) % 50 == 0:
            print(f"Gen {gen+1}: Best = {best_ever.fitness:.2f}")
    
    print(f"\nFinal Best: {best_ever.fitness:.2f}")
    return best_ever, history

def evolution_strategy_plus(config):
    """(μ + λ) Evolution Strategy - compete parents and offspring"""
    print(f"\n{'='*60}")
    print(f"EVOLUTION STRATEGY (μ + λ)")
    print(f"{'='*60}")
    
    mu = config['pop_size']
    lambda_ = config['offspring_size']
    
    # Initialize
    population = config['init_func'](num_cities, mu)
    best_ever = min(population, key=lambda i: i.fitness)
    history = [best_ever.fitness]
    
    for gen in range(config['max_gen']):
        offspring = []
        
        # Generate λ offspring from μ parents
        for _ in range(lambda_):
            # Recombination
            if random.random() < 0.7:
                p1, p2 = random.sample(population, 2)
                child_genotype = config['crossover'](p1.genotype, p2.genotype)
            else:
                parent = random.choice(population)
                child_genotype = parent.genotype[:]
            
            # Mutation
            if random.random() < 0.5:
                child_genotype = config['mutation'](child_genotype)
            else:
                child_genotype = inversion_mutation(child_genotype)
            
            fitness = evaluate_tour(child_genotype)
            offspring.append(Individual(child_genotype, fitness))
        
        # Select best μ from μ+λ (plus selection)
        population.extend(offspring)
        population = sorted(population, key=lambda i: i.fitness)[:mu]
        
        current_best = population[0]
        if current_best.fitness < best_ever.fitness:
            best_ever = current_best
        
        history.append(best_ever.fitness)
        
        if (gen + 1) % 50 == 0:
            print(f"Gen {gen+1}: Best = {best_ever.fitness:.2f}")
    
    print(f"\nFinal Best: {best_ever.fitness:.2f}")
    return best_ever, history

In [32]:
def run_experiments():
    """Run and compare different EA approaches"""
    
    # Base configuration
    base_config = {
        'pop_size': 100,
        'offspring_size': 50,
        'max_gen': 200,
        'mutation_rate': 0.5,
        'mutation': inversion_mutation,
        'crossover': order_crossover,
        'selection': uniform_selection,
        'init_func': mixed_initialization
    }
    
    results = {}
    
    # Experiment 1: (μ, λ)-ES
    print("\n" + "="*60)
    print("EXPERIMENT 1: (μ, λ) Evolution Strategy")
    print("="*60)
    config4 = base_config.copy()
    config4['offspring_size'] = 200  # λ > μ for comma
    best, history = evolution_strategy_comma(config4)
    results['(μ,λ)-ES'] = (best, history)
    
    # Experiment 2: (μ + λ)-ES
    print("\n" + "="*60)
    print("EXPERIMENT 2: (μ + λ) Evolution Strategy")
    print("="*60)
    best, history = evolution_strategy_plus(base_config)
    results['(μ+λ)-ES'] = (best, history)
    
    # Summary
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    for name, (best, history) in sorted(results.items(), key=lambda x: x[1][0].fitness):
        print(f"{name:25} Final Best: {best.fitness:.2f}")

In [33]:
run_experiments()


EXPERIMENT 1: (μ, λ) Evolution Strategy

EVOLUTION STRATEGY (μ, λ)
Gen 50: Best = 1753.33
Gen 100: Best = 1753.33
Gen 150: Best = 1753.33
Gen 200: Best = 1753.33

Final Best: 1753.33

EXPERIMENT 2: (μ + λ) Evolution Strategy

EVOLUTION STRATEGY (μ + λ)
Gen 50: Best = 1764.55
Gen 100: Best = 1764.55
Gen 150: Best = 1764.55
Gen 200: Best = 1764.55

Final Best: 1764.55

SUMMARY
(μ,λ)-ES                  Final Best: 1753.33
(μ+λ)-ES                  Final Best: 1764.55
